In [14]:
require 'redis'
require 'date'
require 'descriptive_statistics'

redis = Redis.new(host: "redis", port: 6379, db: 0)
class String

  def to_table_name
    
    self.gsub(/\W/, '')
  end

end

class Date
  def granule(granularity)
    granularities = [:year, :month, :day]
    idx =  granularities.index(granularity)
    granularities[0..idx].map{|g| eval "self.#{g}" }
  end
end

class Array

  def to_dot(filename = 'digraph.dot')
     File.open(filename, 'w') do |f|
       f.puts "digraph {"
       self.map{|e| f.puts "#{e[0]} -> #{e[1]};"}
       f.puts "}"
     end
  end

  def <(array)
    self.each_with_index do |i, idx|
      if i < array[idx] then  return true end
      if i > array[idx] then  return false end
    end
    return false
  end

   
  def devstd()
    m = self.mean
    sum = 0.0
    freq = Hash.new(0)
    self.each { |x| freq[x] += 1 }
    self.each {|v| sum += (v-m)**2 *freq[v]/freq.size }
    Math.sqrt(sum/self.size)
  end

          
end
      
      
def split(params = {})
  p = {verbose: false}.merge(params)
  series  = signal_series(p)
  total_sd = series.devstd
  total_mean = series.mean
  
  # epsilon=thr/n
  thr=5.0
  n= series.sum
  epsilon = thr/n
  
  puts "n: #{n}"
  puts "thr: #{thr}"
  puts "epsilon: #{epsilon}"
  puts "Series: #{series}"
  puts "total sd: #{total_sd}"

  if p[:verbose] then puts "#{Time.now}: starting split drug: #{p[:drug]} adr:  #{p[:adr]} " end
  if p[:verbose]
    puts "#{Time.now}: series"
    series.each{|s| puts s}
  end
    
 edges = (0..series.size - 1).to_a.select { |i|
    series_values =  series[0..i].map{|j| j[1]}
    if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [0,i + 1]}
  edges += (0..series.size - 1).to_a.select { |i|
    series_values =  series[i..series.size - 1].map{|j| j[1]}
   if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [i + 1, series.size + 1]}
  edges +=  (0..series.size - 1).to_a.product((0..series.size - 1).to_a).select{|i| i[0] < i[1]}.select{|i|
    series_values =  series[i[0]..i[1] - 1].map{|j| j[1]}
    if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [i[0] + 1, i[1] + 1 ]}
    

   shp(edges)
end

def shp(edges)
  t = edges.flatten.max
  shp = (0..t).to_a.map{|_| Float::INFINITY}; shp[0] = 0
  pre = (0..t).to_a.map{|_| nil}
  edges.sort{|a,b| a <=> b}.each{|e| if shp[e[1]] > shp[e[0]] + 1 then shp[e[1]] = shp[e[0]] + 1; pre[e[1]] = e[0] end}
  r = [t]; while !pre[r[-1]].nil? do  r.push(pre[r[-1]]) end
  r.reverse
end          

  
def init_temporal_values(params = {})
  (params[:start_date]..params[:end_date]).to_a.map{|ts| ts.granule(params[:granularity]) }.uniq
end  
  
  
SOC_REDIS = redis  

def signal_series(params = {})
  p = {
       redis: SOC_REDIS,
       adr_level: :soc,
       granularity: :month,
       vts:  [2008, 1],
       vte: [2017,12]
  }.merge(params)
  
  # drug, adr, is_soc, granularity  = p[:drug], p[:adr], p[:soc?], p[:granularity]
  tv = init_temporal_values(start_date: Date.new(p[:vts][0],p[:vts][1],1), 
                       end_date: Date.new(p[:vte][0],p[:vte][1],1), 
                       granularity: :month)
  r = tv.map do |y,m|
                          key = {incremental: false, year: y, month: m, drug: p[:drug], p[:adr_level] => p[:adr] }  
                          v =  p[:redis].get(key)
                          v.nil? ? 0 :  v.to_i
                       end
  p[:granularity] == :year ?  r.each_slice(12).to_a.map{|i| i.reduce(:+)} : r
  
end

<main>:121: warning: already initialized constant SOC_REDIS
<main>:123: warning: previous definition of SOC_REDIS was here


:signal_series

In [4]:
#Obtain all drug-adr pairs
pairs = redis.keys({incremental: false, year: 2017, month: 12, drug: "*", soc: "*"}.to_s); nil
pairs = pairs.map{|p| h = (eval p); {drug: h[:drug], adr: h[:soc]} }.uniq

#for each drug-adr pair compute TEDAR extraction of homogenous intervals and write intervals to file
pairs.each{|p| 
  split=split(p.merge({granularity: :month, verbose: false}))
  s= split.map { |i| i.to_s }.join(",")
  File.write("split_TEDAR.txt", p[:drug]+"\t"+p[:adr]+"\t"+s+"\n", mode: "a")
}

[{:drug=>"AMIODARONE CLORIDRATO", :adr=>"Blood and lymphatic system disorders"}, {:drug=>"DESAMETASONE FOSFATO SODICO", :adr=>"Immune system disorders"}, {:drug=>"QUETIAPINA FUMARATO", :adr=>"Skin and subcutaneous tissue disorders"}, {:drug=>"LOPERAMIDE CLORIDRATO", :adr=>"Skin and subcutaneous tissue disorders"}, {:drug=>"CLOZAPINA", :adr=>"Gastrointestinal disorders"}, {:drug=>"OLANZAPINA PAMOATO MONOIDRATO", :adr=>"Infections and infestations"}, {:drug=>"BACLOFENE", :adr=>"Investigations"}, {:drug=>"MANNITOLO", :adr=>"Blood and lymphatic system disorders"}, {:drug=>"RUXOLITINIB FOSFATO", :adr=>"Metabolism and nutrition disorders"}, {:drug=>"BROMEXINA CLORIDRATO", :adr=>"Respiratory, thoracic and mediastinal disorders"}, {:drug=>"DONEPEZIL", :adr=>"Cardiac disorders"}, {:drug=>"ATENOLOLO", :adr=>"Ear and labyrinth disorders"}, {:drug=>"RIFAMPICINA", :adr=>"Blood and lymphatic system disorders"}, {:drug=>"POTASSIO CANRENOATO", :adr=>"Vascular disorders"}, {:drug=>"PACLITAXEL", :adr=>"

In [15]:
#Compute TEDAR only on specific drug-adr pairs reported in file "drug_adr_sample.txt"

File.readlines('drug_adr_sample.txt').each do |line|
  array=line.strip.split("\t")
  pairs = { :drug => array[0], :soc => array[1] } 
  pairs = redis.keys({incremental: true, year: 2017, month: 12, drug: pairs[:drug].to_s, soc: pairs[:soc].to_s}.to_s); nil
  pairs = pairs.map{|p| h = (eval p); {drug: h[:drug], adr: h[:soc]} }.uniq
  pairs.each { |p|
    puts "\nNew association"
    puts p
    puts split(p.merge({granularity: :month, verbose: false}))
    File.write("sample_result.txt", p[:drug]+"\t"+p[:adr]+"\t"+split(p.merge({granularity: :month, verbose: false})).to_s+"\n", mode: "a")
}
end


New association
{:drug=>"ACICLOVIR", :adr=>"Immune system disorders"}
n: 34
thr: 5.0
epsilon: 0.14705882352941177
Series: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 4, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0]
total sd: 1.162155298689295
[0, 1, 70, 73, 90, 91, 112, 113, 121]
n: 34
thr: 5.0
epsilon: 0.14705882352941177
Series: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 4, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0]
total sd: 1.162155298689295

New asso

["ACICLOVIR\tImmune system disorders\n", "ACICLOVIR\tCongenital, familial and genetic disorders\n", "TERBINAFINA CLORIDRATO\tNervous system disorders\n", "ZOPICLONE\tGastrointestinal disorders\n", "IODIO\tGeneral disorders and administration site conditions\n", "TRANDOLAPRIL\tGeneral disorders and administration site conditions\n", "DIMETICONE\tSkin and subcutaneous tissue disorders"]